# gmxapi workshop
## Goal: Simulation-Analysis loop



DOI: [10.1371/journal.pcbi.1009835](https://doi.org/10.1371/journal.pcbi.1009835)

# gmxapi Python package for GROMACS

* https://manual.gromacs.org/current/gmxapi/index.html
* https://pypi.org/project/gmxapi/

## Literature

* https://doi.org/10.1093/bioinformatics/bty484
* https://doi.org/10.1371/journal.pcbi.1009835

## This material

https://github.com/kassonlab/gmxapi-tutorials/

Funnelweb spider peptide structure files from
* Sorin and Pande, 2005; doi:10.1529/biophysj.104.051938
(used with permission).

## Issue tracking

https://gitlab.com/gromacs/gromacs/-/issues?label_name%5B%5D=gmxapi%3A%3APython

## Contribute

https://gitlab.com/gromacs/gromacs/-/tree/master/python_packaging/src/gmxapi

## Support

### Online docs

* quick start: https://manual.gromacs.org/current/gmxapi/userguide/usage.html
* reference: https://manual.gromacs.org/current/gmxapi/userguide/pythonreference.html

### User forum

https://gromacs.bioexcel.eu/tags/c/gromacs-user-forum/5/gmxapi

### Built-in documentation

* from the shell: `$ pydoc gmxapi`
* from an interactive interpreter:
```python
>>> import gmxapi
>>> help(gmxapi)
```

# Part 1: Introduction to gmxapi

This material requires GROMACS 2022 and gmxapi 0.3. (See [README](https://github.com/kassonlab/gmxapi-tutorials) for more)

In the simplest case, you should be able to activate your GROMACS installation (such as with `module load gromacs` or `source /path/to/gromacs/bin/GMXRC`) and then `pip install gmxapi` into a Python venv.

In [ ]:
import gmxapi as gmx

In [ ]:
gmx.version.api_is_at_least(0, 3)

In [ ]:
# Assumes the tutorial material is in `./AdvancedGromacsCourse/gmxapi-tutorials`.
# Check!
import os
from pathlib import Path
notebook_dir = Path(os.getcwd())
tutorials_dir = notebook_dir.parent
input_dir = tutorials_dir / 'input_files' / 'fs-peptide'
assert input_dir.exists()

## First steps: gmxapi syntax

Commands are used to declare work and get handles to the results of that work.

Work is not performed until the results are needed or requested explicitly.

In [ ]:
cmd = gmx.logical_not(False)

In [ ]:
cmd

In [ ]:
cmd.result()

"Wide" inputs imply parallel data flow pipelines.

In [ ]:
gmx.logical_not([False, True, True]).result()

More complex operations with multiple outputs present collections of output "Futures".

You can define such operations by writing a Python function and decorating it with `function_wrapper`.

In [ ]:
help(gmx.function_wrapper)

Get the lists of keys and values of a dictionary.

```python
(obj: dict) -> (keys: List, values: List)
```

In [ ]:
@gmx.function_wrapper(output={'keys_list': list, 'values_list': list})
def extract_dict(obj: dict, output):
    output.keys_list = list(obj.keys())
    output.values_list = list(obj.values())

In [ ]:
cmd = extract_dict({'spam': False, 'eggs': True})

In [ ]:
# Let's use tab completion to explore this a bit.
cmd.
#cmd.output

In [ ]:
cmd.output.keys_list.result()

Parallel inputs mean parallel outputs.

In [ ]:
output_keys = extract_dict([{'spam': False, 'eggs': True}]*2).output.keys_list

In [ ]:
output_keys.
#output_keys.description

In [ ]:
output_keys.result()[0].to_list()

## The `commandline` module.

In [ ]:
help(gmx.commandline)
# Note to self: Make the docstrings narrower!

In [ ]:
gmx.commandline.cli_executable()

## Wrapping command line tools

Prepare a molecular model from a PDB file using the `pdb2gmx` GROMACS tool.

In [ ]:
# Confirm inputs exist
assert (input_dir / 'start0.pdb').exists()

In [ ]:
args = ['pdb2gmx', '-ff', 'amber99sb-ildn', '-water', 'tip3p']
input_files = {'-f': os.path.join(input_dir, 'start0.pdb')}
output_files = {
        '-p': 'topol.top',
        '-i': 'posre.itp',
        '-o': 'conf.gro'}
make_top = gmx.commandline_operation('gmx', args, input_files, output_files)

In [ ]:
# optional: force execution
# make_top.run()

In [ ]:
# Confirm inputs exist for the next exercise.
assert os.path.exists(make_top.output.file['-o'].result())
assert os.path.exists(make_top.output.file['-p'].result())
cmd_dir = input_dir
assert os.path.exists(input_dir / 'grompp.mdp')

**STOP!** Before we go further, let's give some thought to where our output files are being generated.

In [ ]:
make_top.output.file['-o'].result()

In [ ]:
os.getcwd()

In [ ]:
list(os.listdir())

In [ ]:
os.makedirs('exercise1', exist_ok=True)

In [ ]:
os.chdir('exercise1')

## gmxapi command on ensemble input
Call the GROMACS MD preprocessor to create a simulation input file. Declare an ensemble simulation workflow starting from the single input file.


In [ ]:
grompp_input_files = {'-f': os.path.join(cmd_dir, 'grompp.mdp'),
                      '-c': make_top.output.file['-o'],
                      '-p': make_top.output.file['-p']}

# make array of inputs
N = 40

grompp = gmx.commandline_operation(
    'gmx', ['grompp'],
    input_files = [grompp_input_files] * N,
    output_files = {'-o': 'run.tpr'})
tpr_input = grompp.output.file['-o'].result()

In [ ]:
input_list = gmx.read_tpr(tpr_input)

In [ ]:
input_list.output

In [ ]:
input_list.output.parameters.result()

In [ ]:
input_list.output

In [ ]:
md = gmx.mdrun(input_list, runtime_args={'-maxh': '0.01'})

In [ ]:
md.output.ensemble_width

In [ ]:
md.output.trajectory

In [ ]:
md.run()

print(md.output.trajectory.result())

## Didn't work?

No problem. We'll come back to that in **part 2**.

Let's just do the first one.

In [ ]:
tpr_file = grompp.output.file['-o'].result()[0]

In [ ]:
tpr_file

In [ ]:
md = gmx.mdrun(gmx.read_tpr(tpr_file), runtime_args={'-maxh': '0.001', '-nt': '4'})
md.run()

In [ ]:
md.output.parameters.result()

# End of part 1

## Additional exercises

For part 2, we are going to need some utility functions that we can use in a workflow.

Use `gmx.function_wrapper` to define operations for
* `less_than`
* `numpy_min`
* `xvg_to_array`

# Helper functions

In [ ]:
import logging
import typing

from gmxapi import function_wrapper

def less_than(lhs: typing.SupportsFloat, rhs: typing.SupportsFloat):
    """Compare the left-hand-side to the right-hand-side.

    Follows the Numpy logic for normalizing the numeric types of *lhs* and *rhs*.
    """
    import numpy as np
    dtype = int
    if any(isinstance(operand, float) for operand in (lhs, rhs)):
        dtype = float
    elif all(isinstance(operand, typing.SupportsFloat) for operand in (lhs, rhs)):
        if type(np.asarray([lhs, rhs])[0].item()) is float:
            dtype = float
    elif any(isinstance(operand, gmxapi.abc.Future) for operand in (lhs, rhs)):
        for item in (lhs, rhs):
            if hasattr(item, 'dtype'):
                if issubclass(item.dtype, (float, np.float)):
                    dtype = float
            elif hasattr(item, 'description'):
                if issubclass(item.description.dtype, (float, np.float)):
                    dtype = float
    else:
        raise UsageError(f'No handling for [{repr(lhs)}, {repr(rhs)}].')

    if dtype is int:
        def _less_than(lhs: int, rhs: int) -> bool:
            return lhs < rhs
    elif dtype is float:
        def _less_than(lhs: float, rhs: float) -> bool:
            return lhs < rhs
    else:
        raise UsageError('Operation only supports standard numeric types.')
    return function_wrapper()(_less_than)(lhs=lhs, rhs=rhs)


@function_wrapper()
def _numpy_min_float(data: gmx.NDArray) -> float:
    import numpy as np
    logging.info(f'Looking for minimum in {data}')
    return float(np.min(data._values))


def numeric_min(array):
    """Find the minimum value in an array.
    """
    return _numpy_min_float(data=array)


@gmx.function_wrapper(output={'data': gmx.NDArray})
def xvg_to_array(path: str, output):
    """Get an NxM array from a GROMACS xvg data file.

    For energy output, N is the number of samples, and the first of M
    columns contains simulation time values.
    """
    import numpy
    logging.info(f'Reading xvg file {path}.')
    data = numpy.genfromtxt(path, comments='@', skip_header=14)
    logging.info(f'Read array shape {data.shape} from {path}.')
    if len(data.shape) == 1:
        # Trajectory was too short. Only a single line was read.
        assert data.shape[0] == 2
        data = data.reshape((1, 2))
    assert len(data.shape) == 2
    output.data = data[:, 1]



# Part 2: Simulation-Analysis loop

In [ ]:
while not str(os.getcwd()).endswith('gmxapi-introduction'):
    os.chdir('..')
os.getcwd()

In [ ]:
os.makedirs('exercise2', exist_ok=True)
os.chdir('exercise2')

In [ ]:
os.getcwd()

## Prepare inputs

### Prepare a molecular model from a PDB file using the `pdb2gmx` GROMACS tool.

In [ ]:
# Confirm inputs exist
assert (input_dir / 'start0.pdb').exists()

In [ ]:
args = ['pdb2gmx', '-ff', 'amber99sb-ildn', '-water', 'tip3p']
input_files = {'-f': os.path.join(input_dir, 'start0.pdb')}
output_files = {
        '-p': 'topol.top',
        '-i': 'posre.itp',
        '-o': 'conf.gro'}
make_top = gmx.commandline_operation('gmx', args, input_files, output_files)

### Prepare the simulation input

Call the GROMACS MD preprocessor to create a simulation input file.

In [ ]:
# Confirm inputs exist.
# assert os.path.exists(make_top.output.file['-o'].result())
# assert os.path.exists(make_top.output.file['-p'].result())
# assert os.path.exists(input_dir / 'grompp.mdp')

In [ ]:
cmd_dir = input_dir

grompp_input_files = {'-f': os.path.join(cmd_dir, 'grompp.mdp'),
                      '-c': make_top.output.file['-o'],
                      '-p': make_top.output.file['-p']}

grompp = gmx.commandline_operation(
    'gmx', ['grompp'],
    input_files = [grompp_input_files],
    output_files = {'-o': 'run.tpr'})
tpr_input = grompp.output.file['-o'].result()

In [ ]:
input_list = gmx.read_tpr(tpr_input)

#### Inspect

In [ ]:
input_list.output.parameters.result()

#### Adjust input parameters

In [ ]:
input_list = gmx.modify_input(input_list, parameters={'nstxout': 100})

## Looping and custom operations

In [ ]:
reference_struct = input_dir / 'ref.pdb'
assert reference_struct.exists()

In [ ]:
subgraph = gmx.subgraph(
    variables={
        'found_native': False,
        'checkpoint': '',
        'min_rms': 1e6})
with subgraph:
    md = gmx.mdrun(
        input_list,
        runtime_args={
            '-cpi': subgraph.checkpoint,
            '-maxh': '0.001',
            '-noappend': None,
            '-nt': '8'
        })

    subgraph.checkpoint = md.output.checkpoint
    rmsd = gmx.commandline_operation(
        'gmx', ['rms'],
        input_files={
            '-s': reference_struct,
            '-f': md.output.trajectory},
        output_files={'-o': 'rmsd.xvg'},
        stdin='Backbone Backbone\n'
    )
    subgraph.min_rms = numeric_min(
        xvg_to_array(rmsd.output.file['-o']).output.data).output.data
    subgraph.found_native = less_than(lhs=subgraph.min_rms, rhs=0.3).output.data

folding_loop = gmx.while_loop(
    operation=subgraph,
    condition=gmx.logical_not(subgraph.found_native))()
print('Beginning folding_loop.')
folding_loop.run()
print(f'Finished folding_loop. min_rms: {folding_loop.output.min_rms.result()}')

# Exercise 3: Ensemble simulation
Use the built-in mpi4py ensemble executor.

In [ ]:
while not str(os.getcwd()).endswith('gmxapi-introduction'):
    os.chdir('..')
os.getcwd()

In [ ]:
os.makedirs('exercise3', exist_ok=True)
os.chdir('exercise3')

In [ ]:
script_dir = input_dir.parent.parent / 'examples'
example = script_dir / 'fs-peptide.py'
assert os.path.exists(example)
print(example)

This exercise will be done from the command line.

We can try doing mpirun from this notebook. Copy the script path from above to the appropriate place below.

The script path (printed above) needs to be launched in an MPI context.

In the simplest case, a 10-member ensemble would be launched like the following, but you may need a different launch method (e.g. `srun`) depending on your HPC environment.

```bash
mpiexec -n 10 -m `which python` -m mpi4py /path/to/fs-peptide.py
```

You are strongly encouraged to examine the `fs-peptide.py` script before running it.